In [0]:
pip install transformers sentencepiece torch --upgrade langchain pydantic==1.8 mlflow

Python interpreter will be restarted.
  Using cached transformers-4.33.0-py3-none-any.whl (7.6 MB)
  Using cached sentencepiece-0.1.99-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached torch-2.0.1-cp39-cp39-manylinux1_x86_64.whl (619.9 MB)
  Using cached langchain-0.0.281-py3-none-any.whl (1.6 MB)
  Using cached pydantic-1.8-cp39-cp39-manylinux2014_x86_64.whl (11.3 MB)
  Using cached mlflow-2.6.0-py3-none-any.whl (18.3 MB)
  Using cached PyYAML-6.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (738 kB)
  Using cached safetensors-0.3.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached regex-2023.8.8-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (771 kB)
  Using cached huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
  Using cached tokenizers-0.13.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Using cached nvidia_cuda_nvrtc_cu11-

In [0]:
import mlflow
import mlflow.huggingface
import pandas as pd
from transformers import pipeline
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
%run "/Users/mohamed.zahid@simpleenergy.in/LLM Sentiment Model"

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
df

,date,comment_text,sentiment_label,sentiment_score
0,2023-05-13,why this looks like ather 450x,negative,0.649711
1,2023-05-12,No competition with ola 😅,neutral,0.514803
2,2023-04-25,If everything goes well this will be a big hit...,positive,0.795603
3,2023-03-31,One of the oldest auto channel in YouTube,positive,0.447022
4,2023-02-23,1and hlf yr bck I was thinking to tk franchise...,negative,0.397321
...,...,...,...,...
628,2023-05-23,Nice🎉,positive,0.644901
629,2023-05-23,Cool,positive,0.491287
630,2023-05-23,Hum first,neutral,0.476607
631,2023-05-23,1st..😂🎉,positive,0.578617


In [0]:
# Initialize an MLflow experiment
mlflow.set_experiment("/Users/mohamed.zahid@simpleenergy.in/sentiment_analysis")

# Define the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")

# Function to classify a single row and return the label with the highest score
def classify_and_get_highest_label(row):
    output = classifier(row['comment_text'], feature_labels, multi_label=False)
    index_of_highest_score = output["scores"].index(max(output["scores"]))
    return output["labels"][index_of_highest_score]

feature_labels = [
    "booking_cancellation_process",
    "range",
    "battery",
    "reliability_quality",
    "communication_by_company",
    "comparison",
    "price",
    "sales_service",
    "delivery_time",
    "performance",
    "design",
    "vehicle_issues_faults"
]

with mlflow.start_run():
    # Log parameters
    mlflow.log_params({"feature_labels": feature_labels})
    
    # Apply the classification function to each row and append the result to a new 'features' column
    df['features'] = df.apply(classify_and_get_highest_label, axis=1)

    # Log artifacts
    mlflow.log_artifacts("/dbfs/FileStore")

    # Save the model for model registry
    mlflow.huggingface.save_model(
        model_name="sentiment_analysis_model",
        path="dbfs:/FileStore",
        registered_model_name="sentiment_analysis_model_v1"
    )

df

In [0]:
display(df)

date,comment_text,sentiment_label,sentiment_score,features
2023-05-13,why this looks like ather 450x,negative,0.6497108340263367,comparison
2023-05-12,No competition with ola 😅,neutral,0.5148032903671265,communication_by_company
2023-04-25,If everything goes well this will be a big hit in EV market for few years,positive,0.7956030368804932,comparison
2023-03-31,One of the oldest auto channel in YouTube,positive,0.4470221698284149,design
2023-02-23,"1and hlf yr bck I was thinking to tk franchise showroom of this n I clld them, they were asking for advance.. thnk god I did not pay.",negative,0.39732062816619873,communication_by_company
2023-02-21,Modified Ather 😄,positive,0.46343493461608887,comparison
2023-02-19,I am waiting for a long time for this scooty pls simple one do fast and open showroom in Kolkata,neutral,0.4584501087665558,communication_by_company
2023-02-18,Romovable battery is an advantage,positive,0.6531673073768616,battery
2023-02-16,Bro make a video hill climb test.,neutral,0.5588948130607605,performance
2023-02-14,Looks like ather,neutral,0.5686222910881042,comparison


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
# Initialize an MLflow experiment
mlflow.set_experiment("/Users/mohamed.zahid@simpleenergy.in/sentiment_analysis")

# Define the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")

# Function to classify a single row and return the label with the highest score
def classify_and_get_highest_label(row):
    output = classifier(row['comment_text'], feature_labels, multi_label=False)
    index_of_highest_score = output["scores"].index(max(output["scores"]))
    return output["labels"][index_of_highest_score]

feature_labels = [
    "booking_cancellation_process",
    "range",
    "battery",
    "reliability_quality",
    "communication_by_company",
    "comparison",
    "price",
    "sales_service",
    "delivery_time",
    "performance",
    "design",
    "vehicle_issues_faults"
]

with mlflow.start_run():
    # Log parameters
    mlflow.log_params({"feature_labels": feature_labels})
    df['features'] = df.apply(classify_and_get_highest_label, axis=1)

    # Log artifacts
    mlflow.log_artifacts("dbfs:/FileStore")

    # Save the model for model registry
    mlflow.huggingface.save_model(
        model_name="sentiment_analysis_model",
        path="dbfs:/FileStore",
        registered_model_name="sentiment_analysis_model_v1"
    )
    
df


In [0]:
# Initialize an MLflow experiment
mlflow.set_experiment("/Users/mohamed.zahid@simpleenergy.in/sentiment_analysis")

# Define the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")

# Function to classify a single row and return the label with the highest score
def classify_and_get_highest_label(row):
    output = classifier(row['comment_text'], feature_labels, multi_label=False)
    index_of_highest_score = output["scores"].index(max(output["scores"]))
    return output["labels"][index_of_highest_score]

# Apply the classification function to each row and append the result to a new 'features' column
#candidate_labels = ["politics", "economy", "entertainment", "environment"]
feature_labels = [
    "booking_cancellation_process",
    "range",
    "battery",
    "reliability_quality",
    "communication_by_company",
    "comparison",
    "price",
    "sales_service",
    "delivery_time",
    "performance",
    "design",
    "vehicle_issues_faults"
]

with mlflow.start_run():
    # Log parameters
    mlflow.log_params({"feature_labels": feature_labels})
    # Load your DataFrame 'df'
    # df = pd.read_csv("your_data.csv")

    df['features'] = df.apply(classify_and_get_highest_label, axis=1)

    # Log artifacts
    mlflow.log_artifacts("dbfs:/FileStore")
    
df

In [0]:
# Initialize an MLflow experiment
mlflow.set_experiment("/Users/mohamed.zahid@simpleenergy.in/sentiment_analysis")

# Define the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")

# Function to classify a single row and return the label with the highest score
def classify_and_get_highest_label(row):
    output = classifier(row['comment_text'], feature_labels, multi_label=False)
    index_of_highest_score = output["scores"].index(max(output["scores"]))
    return output["labels"][index_of_highest_score]

# Apply the classification function to each row and append the result to a new 'features' column
#candidate_labels = ["politics", "economy", "entertainment", "environment"]
feature_labels = [
    "booking_cancellation_process",
    "range",
    "battery",
    "reliability_quality",
    "communication_by_company",
    "comparison",
    "price",
    "sales_service",
    "delivery_time",
    "performance",
    "design",
    "vehicle_issues_faults"
]

with mlflow.start_run():
    # Log parameters
    mlflow.log_params({"feature_labels": feature_labels})
    # Load your DataFrame 'df'
    # df = pd.read_csv("your_data.csv")

    df['features'] = df.apply(classify_and_get_highest_label, axis=1)

df